In [2]:
import os
os.chdir('../')
os.getcwd()
mainDir = os.getcwd()
print('current working directory: ', mainDir)

current working directory:  /home/minje/springbank


In [3]:
mainDir = os.getcwd()
print('current working directory: ', mainDir)

current working directory:  /home/minje/springbank


In [4]:
from tools.BinanceApiWrappers import BinanceApiWrappers
from tools.LogicFunctions import LogicFunctions

In [5]:
binance = BinanceApiWrappers()
lf = LogicFunctions()

In [14]:
binance.getAvailableCoinPairLists()

['BTCUSDT',
 'ETHUSDT',
 'BNBUSDT',
 'NEOUSDT',
 'LTCUSDT',
 'QTUMUSDT',
 'ADAUSDT',
 'XRPUSDT',
 'EOSUSDT',
 'IOTAUSDT',
 'XLMUSDT',
 'ONTUSDT',
 'TRXUSDT',
 'ETCUSDT',
 'ICXUSDT',
 'VETUSDT',
 'LINKUSDT',
 'WAVESUSDT',
 'BTTUSDT',
 'HOTUSDT',
 'ZILUSDT',
 'ZRXUSDT',
 'BATUSDT',
 'XMRUSDT',
 'ZECUSDT',
 'IOSTUSDT',
 'CELRUSDT',
 'DASHUSDT',
 'OMGUSDT',
 'THETAUSDT',
 'ENJUSDT',
 'MATICUSDT',
 'ATOMUSDT',
 'ONEUSDT',
 'FTMUSDT',
 'ALGOUSDT',
 'DOGEUSDT',
 'ANKRUSDT',
 'MTLUSDT',
 'TOMOUSDT',
 'DENTUSDT',
 'CVCUSDT',
 'CHZUSDT',
 'BANDUSDT',
 'XTZUSDT',
 'RENUSDT',
 'RVNUSDT',
 'HBARUSDT',
 'NKNUSDT',
 'KAVAUSDT',
 'ARPAUSDT',
 'IOTXUSDT',
 'RLCUSDT',
 'BCHUSDT',
 'OGNUSDT',
 'BTSUSDT',
 'COTIUSDT',
 'SOLUSDT',
 'CTSIUSDT',
 'CHRUSDT',
 'STMXUSDT',
 'KNCUSDT',
 'LRCUSDT',
 'COMPUSDT',
 'SCUSDT',
 'ZENUSDT',
 'SNXUSDT',
 'DGBUSDT',
 'SXPUSDT',
 'MKRUSDT',
 'STORJUSDT',
 'MANAUSDT',
 'YFIUSDT',
 'BALUSDT',
 'BLZUSDT',
 'SRMUSDT',
 'CRVUSDT',
 'SANDUSDT',
 'OCEANUSDT',
 'DOTUSDT',
 'LUNAUS

In [71]:
import json
import requests
from decimal import Decimal as dec


def fetch_price_difference():
    spot_req_result = requests.get("https://www.binance.com/api/v3/ticker/bookTicker").text #api 2회 소모
    futures_req_result = requests.get("https://www.binance.com/fapi/v1/ticker/bookTicker").text #api 2회 소모, 총 4회 소모
    
    spot_orderbook = json.loads(spot_req_result)
    futures_orderbook = json.loads(futures_req_result)

    spot_orderbook_dict = {
        element.get('symbol'):{
            'bidPrice_spot': element.get('bidPrice'),
            'bidQty_spot': element.get('bidQty'), 
            'askPrice_spot': element.get('askPrice'),
            'askQty_spot': element.get('askQty'),
        }for element in spot_orderbook if element.get('symbol').endswith('USDT')}

    futures_orderbook_dict = {
        element.get('symbol'):{
            'bidPrice_futures': element.get('bidPrice'),
            'bidQty_futures': element.get('bidQty'), 
            'askPrice_futures': element.get('askPrice'),
            'askQty_futures': element.get('askQty'),
        }for element in futures_orderbook if element.get('symbol').endswith('USDT')}
    
    trackingList = (spot_orderbook_dict.keys() & futures_orderbook_dict.keys())

    priceDict = {coins: {'symbol': coins,
            'bidPrice_spot': spot_orderbook_dict.get(coins).get('bidPrice_spot'),
            'bidQty_spot': spot_orderbook_dict.get(coins).get('bidQty_spot'), 
            'askPrice_spot': spot_orderbook_dict.get(coins).get('askPrice_spot'),
            'askQty_spot': spot_orderbook_dict.get(coins).get('askQty_spot'),
            'bidPrice_futures': futures_orderbook_dict.get(coins).get('bidPrice_futures'),
            'bidQty_futures': futures_orderbook_dict.get(coins).get('bidQty_futures'), 
            'askPrice_futures': futures_orderbook_dict.get(coins).get('askPrice_futures'),
            'askQty_futures': futures_orderbook_dict.get(coins).get('askQty_futures'),

            'entering_difference': (dec(futures_orderbook_dict.get(coins).get('bidPrice_futures')) - dec(spot_orderbook_dict.get(coins).get('askPrice_spot'))),
            'leaving_difference': (dec(spot_orderbook_dict.get(coins).get('bidPrice_spot')) - dec(futures_orderbook_dict.get(coins).get('askPrice_futures')))
            }for coins in trackingList} 

    return priceDict

from collections import defaultdict
import time
import numpy as np

differenceDict = defaultdict(lambda: {'entering':[],'leaving':[]}, {})

for i in range(5):
    priceDict = fetch_price_difference()
    for coins in priceDict.values():
        differenceDict[coins.get('symbol')].get('entering').append(coins['entering_difference'])
        differenceDict[coins.get('symbol')].get('leaving').append(coins['leaving_difference'])
    #time.sleep(1)

resultDict = defaultdict(lambda: {'entering':'','leaving':''}, {})
for coins in dict(differenceDict):
    resultDict[coins]['entering'] = (np.mean(differenceDict[coins]['entering']), np.std(differenceDict[coins]['entering']))
    resultDict[coins]['leaving'] = (np.mean(differenceDict[coins]['leaving']), np.std(differenceDict[coins]['leaving']))

dict(resultDict)['LUNAUSDT']

{'entering': (Decimal('-0.09060000'),
  Decimal('0.01228983319658977308123149923')),
 'leaving': (Decimal('0.03540000'),
  Decimal('0.01710672382427447736692720054'))}

In [19]:
from decimal import Decimal as dec
import numpy
import time

def movgingaverage(symbol):
    candleData = lf.binanceApiWrappers.apiManager.futureBinance.fapiPublicGetKlines(
        {'symbol': symbol, 'interval': '1m'})[-21:-1]
    
    closeValues = list(map(lambda element: float(element[4]), candleData))
    simple_average = numpy.mean(closeValues)
    stdev = numpy.std(closeValues)

    volume_weighted_sum = dec('0')
    total_volume = dec('0')
    for items in candleData:
        total_volume += dec(items[5])
        volume_weighted_sum += dec(items[4]) * dec(items[5])

    return {
        'ma': str(simple_average), 
        'volume_average': str(volume_weighted_sum/total_volume),
        'stdev': str(stdev), 
        'boll3.3_up': str(simple_average + 3.3*stdev), 
        'boll3.3_down': str(simple_average - 3.3*stdev)}

movgingaverage('LUNAUSDT')

def boll_arbitrage(symbol):

    status = 'NONE' # NONE, SOLD, BOUGHT
    print('시작', 'STATUS: ', status)

    while True:

        bandInfo = movgingaverage(symbol)

        futures_ticker = lf.binanceApiWrappers.getFutureOrderBookTicker(symbol)
        futures_bid_price = dec(futures_ticker.get('bidPrice'))
        futures_ask_price = dec(futures_ticker.get('askPrice'))

        if status == 'BOUGHT':
            if futures_bid_price > dec(bandInfo.get('ma')):
                lf.binanceApiWrappers.orderFutureSell(symbol, '5')
                status = 'NONE'
                print('샀던 코인을 팔았음')

        elif status == 'SOLD':
            if futures_ask_price < dec(bandInfo.get('ma')):
                lf.binanceApiWrappers.orderFutureBuy(symbol, '5')
                status = 'NONE'
                print('팔았던 코인을 샀음')
        
        else:
            if futures_bid_price > dec(bandInfo.get('boll3.3_up')):
                lf.binanceApiWrappers.orderFutureSell(symbol, '5')
                status = 'SOLD'
                print('볼린저 3.3 위를 돌파해 코인을 팔았음')
            if futures_ask_price < dec(bandInfo.get('boll3.3_down')):
                lf.binanceApiWrappers.orderFutureBuy(symbol, '5')
                status = 'BOUGHT'
                print('볼린저 3.3 아래를 돌파해 코인을 샀음')
        
        time.sleep(1)

In [6]:
boll_arbitrage('LUNAUSDT')

시작 STATUS:  NONE


KeyboardInterrupt: 